# Tensorflow Example

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import mlflow
from dotenv import load_dotenv
import os
import boto3

d:\Miniconda\envs\tiny_cnn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Settting up environment variables

load_dotenv()  

S3_BUCKET = os.getenv("S3_BUCKET")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
TRACKING_URI = os.getenv("TRACKING_URI")

In [3]:
boto3.setup_default_session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

# Prearing the dataset

In [4]:
(ds_train, ds_test), ds_info = tfds.load(
 "mnist",
 split=["train", "test"],
 shuffle_files=True,
 as_supervised=True,
 with_info=True,
)
# Normalizes images: `uint8` -> `float32`
def normalize_img(image, label):
 return tf.cast(image, tf.float32) / 255., label
 
# Train Dataset
ds_train = ds_train.map(
 normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
# Test Dataset
ds_test = ds_test.map(
 normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

# Defining the model

In [5]:
# Define the layers
inputs = tf.keras.Input(shape=(28, 28, 1))
hidden = tf.keras.layers.Flatten()(inputs)
hidden2 = tf.keras.layers.Dense(128, activation="relu")(hidden)
outputs = tf.keras.layers.Dense(10, activation="softmax")(hidden2)
# Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.002)
# Create a Model object
model = tf.keras.Model(inputs, outputs)
# Compile the model
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [11]:
BATCH_SIZE = 128
EPOCHS = 10
LOGGING_STEPS = 10

EXPERIMENT = "MNIST"
RUN_NAME = "Run_6"

In [12]:
mlflow.tensorflow.autolog(every_n_iter=LOGGING_STEPS)
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT)
mlflow.start_run(run_name=RUN_NAME)

<ActiveRun: >

In [13]:
mlflow.get_tracking_uri()

'http://ec2-18-194-137-129.eu-central-1.compute.amazonaws.com:5000'

In [14]:
history = model.fit(ds_train, epochs=EPOCHS, validation_data=ds_test,
    batch_size=BATCH_SIZE,
    use_multiprocessing = True)
mlflow.end_run()

Epoch 1/10
  1/469 [..............................] - ETA: 32s - loss: 0.0103 - accuracy: 0.9922WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0026s vs `on_train_batch_end` time: 0.0039s). Check your callbacks.


469/469 [==============================] - 2s 5ms/step - loss: 0.0162 - accuracy: 0.9949 - val_loss: 0.0886 - val_accuracy: 0.9753
Epoch 2/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0140 - accuracy: 0.9959 - val_loss: 0.0845 - val_accuracy: 0.9772
Epoch 3/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0112 - accuracy: 0.9968 - val_loss: 0.0946 - val_accuracy: 0.9765
Epoch 4/10
469/469 [==============================] - 3s 5ms/step - loss: 0.0092 - accuracy: 0.9974 - val_loss: 0.0917 - val_accuracy: 0.9758
Epoch 5/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0091 - accuracy: 0.9975 - val_loss: 0.0955 - val_accuracy: 0.9778
Epoch 6/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0102 - accuracy: 0.9967 - val_loss: 0.0896 - val_accuracy: 0.9775
Epoch 7/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0083 - accuracy: 0.9973 - val_loss: 0.1082 - val_accuracy: 0.9752
Epoch 8/10
469/46

INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpp69vheeo\model\data\model\assets
